In [15]:
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer


stemmer = PorterStemmer()
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
  tokenized_sentence = [stem(w) for w in tokenized_sentence]

  bag = np.zeros(len(all_words), dtype = np.float32)

  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  return bag


[1. 0. 0. 1.]


#generating the dataset

In [19]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy


with open("/content/intents.json", "r") as f:
  intents = json.load(f)

all_words= []
tags = []
xy = []

for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)

  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)

    xy.append((w, tag))

ignore_words = [",", '.', '!', "?"]
all_words = [stem(word) for word in all_words if word not in ignore_words]
print(all_words)

all_words = sorted(set(all_words))
tags = sorted(set(tags))


X_train = []
y_train = []

for pattern_sentence, tag in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  X_train.append(bag)

  label = tags.index(tag)

  y_train.append(label)

X_train = numpy.array(X_train)
y_train = numpy.array(y_train)


class ChatDataset():
  def __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples

#hyperParameter
batch_size =8
input_size = len(all_words)
hidden_size = 32
output_size = len(tags)
learning_rate = 0.01
epochs = 100


dataset = ChatDataset()
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

['hi', 'how', 'are', 'you', 'is', 'anyon', 'there', 'hello', 'good', 'day', 'what', 'up', 'how', 'are', 'ya', 'heyi', 'whatsup', 'cya', 'see', 'you', 'bye', 'bye', 'see', 'you', 'later', 'goodby', 'i', 'am', 'leav', 'bye', 'have', 'a', 'good', 'day', 'talk', 'to', 'you', 'later', 'tyyl', 'i', 'got', 'to', 'go', 'gtg', 'what', 'is', 'the', 'name', 'of', 'your', 'develop', 'what', 'is', 'the', 'name', 'of', 'your', 'creator', 'what', 'is', 'the', 'name', 'of', 'the', 'develop', 'what', 'is', 'the', 'name', 'of', 'the', 'creator', 'who', 'creat', 'you', 'your', 'develop', 'your', 'creator', 'who', 'are', 'your', 'develop', 'develop', 'you', 'are', 'made', 'by', 'you', 'are', 'made', 'by', 'whom', 'who', 'creat', 'you', 'who', 'creat', 'you', 'creator', 'who', 'made', 'you', 'who', 'design', 'you', 'name', 'your', 'name', 'do', 'you', 'have', 'a', 'name', 'what', 'are', 'you', 'call', 'what', 'is', 'your', 'name', 'what', 'should', 'i', 'call', 'you', 'what', 'your', 'name', 'what', 'are',

#model

In [23]:
class NeuralNet(nn.Module):


  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, output_size)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)
    return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(epochs):

  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)

    output = model(words)
    loss = criterion(output, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch + 1) % 10 == 0:
    print(f'epoch {epoch+1}/{epochs}, loss={loss.item():.4f}')

print(f'final loss, loss={loss.item():.4f}')

model_state = model.state_dict()



epoch 10/100, loss=0.0005
epoch 20/100, loss=0.0004
epoch 30/100, loss=0.0011
epoch 40/100, loss=0.0003
epoch 50/100, loss=0.0000
epoch 60/100, loss=0.0001
epoch 70/100, loss=0.0001
epoch 80/100, loss=0.0000
epoch 90/100, loss=0.0000
epoch 100/100, loss=0.0001
final loss, loss=0.0001


#chatting

In [29]:
import random

model.eval()

bot_name = "sam"
output_list = []
input_list = ['hi how are you', 'what is your name', "tutturu", 'do you have a hostel']
for input_sentence in input_list:

  input_sentence = tokenize(input_sentence)
  X = bag_of_words(input_sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)
  output = model(X)

  _, predicted = torch.max(output, dim = 1)
  tag = tags[predicted.item()]
  probs = torch.softmax(output, dim = 1)
  prob = probs[0][predicted.item()]
  if prob.item() > 0.75:

    for intent in intents['intents']:
      if tag == intent['tag']:
        print(random.choice(intent['responses']))
  else:
    print("I do not understand...")




Good to see you again!
You can call me Rawkush.
I do not understand...
Ch. Brahm Prakash Government Engineering College does  provide hostel facility
